# Dash Testing

In [ ]:
from dash import Dash, html, dcc, Input, Output
import altair as alt
import dash_bootstrap_components as dbc
import pandas as pd
import numpy as np
import matplotlib as plt
from matplotlib.pyplot import figure

import sheworewhat as sww

# alt.data_transformers.disable_max_rows()

closet = sww.closet_df()
worn_df = sww.complete_df(closet)
top_10, heat_df = sww.top_10_df()

def plot_mostworn():
    most_worn = worn_df.nlargest(15, columns="Count")
    closet_comp = alt.Chart(most_worn, title="2023 Most Worn Pieces"
                       ).mark_bar(color="#6ce4d8"
                         ).encode(alt.X("Name",
                                        title="",
                                        axis=alt.Axis(labelAngle=-45), 
                                        sort="-y"),
                                  alt.Y("Count", 
                                        title="Times Worn",
                                        axis=alt.Axis(tickMinStep=1)),
                                  alt.Tooltip("Count")
                                 ).configure_axis(grid=False, domain=False)
                
    return closet_comp.to_html()

def plot_facet():
    
    categories = ["Top", "Bottom", "Full Body", "Outerwear", "Accessory", "Shoes"]

    cat_plots = []

    for i in categories:
        category_worn = worn_df.loc[worn_df["Category"] == i].nlargest(5, columns="Count")

        category_plot = alt.Chart(category_worn, title=f"2023 Most Worn {i}"
                           ).mark_bar(color="#827191",
                           ).encode(alt.X("Name", title="", axis=alt.Axis(labelAngle=-45), sort="-y"),
                                    alt.Y("Count", 
                                    title="# of Times Worn",
                                    axis=alt.Axis(tickMinStep=1)),
                                    alt.Tooltip(["Name", "Count"])
                            ).properties(
                                height=200,
                                width=150
                                )
        cat_plots.append(category_plot)

    # configure altair charts
    row1 = alt.hconcat(cat_plots[0], cat_plots[1], cat_plots[2])
    row2 = alt.hconcat(cat_plots[3], cat_plots[4], cat_plots[5])

    category_plot = alt.vconcat(row1, row2).configure_axis(grid=False, domain=False)
    return category_plot.to_html()

def plot_heatmap(top_10, df, i=0):
    """
    Function for heatmap plot.
    
    Parameters:
    ----------- 
         top_10 : list
            List containing the IDs of the top 10 most worn items.
            
         df : pandas.DataFrame  
         
    Returns:
    --------
        heatplot : altair.Chart
            Heatmap plot for a single item over a single calender year. 
             
    """
    
    # column of day of week for one calender year 
    time_df = pd.DataFrame()
    time_df["Date"] = pd.date_range(df["Date"].min(), periods=365) 
    time_df["Day"] = time_df["Date"].dt.day_name()
    
    
    heatmap_data = df.loc[df["ID"] == top_10[i]] # need to make this dynamic in plot
    item_name = heatmap_data["Brand"].iloc[0] + " " + heatmap_data["Item"].iloc[0]
    
    # isolate item data
    year = pd.merge(time_df, heatmap_data, how="outer", on="Date")
    year["Item"] = year["Item"].replace(np.nan, 0) 
    year["Bool"] = np.where(year["Item"] == 0, 0, 1)

    # this is horrible to read lol
    # wrangling for prettier plotting 
    week = time_df["Date"].dt.isocalendar()
    year["Week"] = week["week"].fillna(52)
    year["Week"] = year["Week"].fillna(52)
    year['First_day'] = year['Date'] - year['Date'].dt.weekday * np.timedelta64(1, 'D')
    week = time_df["Date"].dt.strftime('%m-%d-%y')
    year["Week"] = year["First_day"].dt.strftime('%m-%d')
    year = year[["Date", "Day", "Item", "ID", "Bool", "Week"]]

    weekdays = ["Sunday", "Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Sunday"]

    heat_plot = alt.Chart(year, title=f"{item_name} in 2023").mark_rect(
        stroke="white",
        strokeWidth=3,
        opacity=0.9,
        ).encode(
        alt.X("Week:O", 
              axis = alt.Axis(labelAngle=-60, )
             ), 
        alt.Y("Day", 
              sort=weekdays,
             title=""),
        alt.Color("Bool", 
                scale=alt.Scale(
                    domain=[0,1], 
                    range=["#e0ddd5", "#74a675"]),
                    legend=None
                 ),
        alt.Tooltip(["Date", "Day"])
        ).properties(
        height=200,
        width=600
        ).configure_axis(grid=False, domain=False)
    return heat_plot.to_html()

app = Dash(__name__)
server = app.server

app.title = "She Wore What 2023"

app.layout = dbc.Container(
    [
        html.Br(),
        dbc.Row(
            dbc.Col(
                dbc.Card(
                    dbc.CardBody(
                        html.B("She Wore What 2023"),
                    ), 
                ),
                style={"font-weight": "bold", 
                       "color": "#000000", 
                       "font-family" :"Garamond",
                      "font-size" : "120%"},
                className='text-center mt-4 mb-5'
            )
        ),
        dbc.Row([
            dbc.Col(
                dbc.Card([
                    dbc.CardBody([
                        html.Div([
                            html.Iframe(
                                id='mostworn',
                                style={'border-width': '0', 'width': '100%', 'height': '600px'},
                                srcDoc=plot_heatmap(top_10, heat_df)
                                )
                        ]),
                        html.Div([
                            html.Iframe(
                                id='mostworn2',
                                style={'border-width': '0', 'width': '100%', 'height': '600px'},
                                srcDoc=plot_facet()
                                )
                        ]),
                    ]),
                    
                ])
            ), 
            dbc.Col(
                dbc.Card([
                    dbc.CardBody([
                        html.Div([
                            html.Iframe(
                                id='mostworn3',
                                style={'border-width': '0', 'width': '100%', 'height': '600px'},
                                srcDoc=plot_mostworn()
                                )
                        ]),
                    ])
                ])
            ),
        ]),
        dbc.Col(
            dbc.Row(
                dbc.Card(
                    dbc.CardBody([
                        html.B("Title2")
                    ]),
                )
            ),
        )
    ])

if __name__ == '__main__':
    app.run_server(
        port = 8073,
        debug=False)

In [ ]:
from dash import Dash, html, dcc, Input, Output
import altair as alt
import dash_bootstrap_components as dbc
import pandas as pd
import numpy as np
import matplotlib as plt
from matplotlib.pyplot import figure

import sheworewhat as sww

closet = sww.closet_df()
worn_df = sww.complete_df(closet)
top_10, heat_df = sww.top_10_df()

app = Dash(__name__)
server = app.server

app.title = "She Wore What 2023"

app.layout = dbc.Container(
    [
        html.Br(),
        dbc.Row(
            dbc.Col(
                dbc.Card(
                    dbc.CardBody(
                        html.B("She Wore What 2023"),
                    ), 
                ),
                style={"font-weight": "bold", 
                       "color": "#000000", 
                       "font-family" :"Garamond",
                      "font-size" : "120%"},
                className='text-center mt-4 mb-5'
            )
        ),
        dbc.Row([
            dbc.Col(
                dbc.Card([
                    dbc.CardBody([
                        
                        html.H4("Card title", className="card-title"),
                            html.P(
                            "Some quick example text to build on the card title and "
                            "make up the bulk of the card's content.",
                            className="card-text",
                        ),
                    dbc.Button("Go somewhere", color="primary"),
                        html.Div([
                            html.Iframe(
                                id='mostworn',
                                style={'border-width': '0', 'width': '100%', 'height': '600px'},
                                srcDoc= sww.plot_heatmap(top_10, heat_df).to_html()
                            )
                        ]),
                        dbc.CardFooter("This is the footer")
                    ]),    
                ])
            ), 
            dbc.Col(
                dbc.Card([
                    dbc.CardBody([
                        html.Div([
                            html.Iframe(
                                id='mostworn3',
                                style={'border-width': '0', 'width': '100%', 'height': '600px'},
                                srcDoc = sww.plot_mostworn(worn_df).to_html()
                                )
                        ]),
                    ])
                ])
            ),
        ]),
    ])

if __name__ == '__main__':
    app.run_server(
        port = 8076,
        debug=False)

In [ ]:
from dash import Dash, html, dcc, Input, Output
import altair as alt
import dash_bootstrap_components as dbc
import pandas as pd
import numpy as np
import matplotlib as plt
from matplotlib.pyplot import figure

import sheworewhat as sww

closet = sww.closet_df()
worn_df = sww.complete_df(closet)
top_10, heat_df = sww.top_10_df()

app = Dash(__name__)
server = app.server

app.title = "She Wore What 2023"

app.layout = dbc.Container(
    [
        html.Br(),
        dbc.Row(
            dbc.Col(
                dbc.Card(
                    dbc.CardBody(
                        html.B("She Wore What 2023"),
                    ), 
                ),
                style={"font-weight": "bold", 
                       "color": "#000000", 
                       "font-family" :"Garamond",
                      "font-size" : "120%"},
                className='text-center mt-4 mb-5'
            )
        ),
        dbc.Row([
            dbc.Col(
                dbc.Card([
                    dbc.CardBody([
                        html.H4("Card title", className="card-title"),
                            html.P(
                            "Some quick example text to build on the card title and "
                            "make up the bulk of the card's content.",
                            className="card-text",
                        ),
                        dbc.Button("A Button", color="#e0ddd5"),
                            html.Div([
                                html.Iframe(
                                    id='mostworn',
                                    style={'border-width': '0', 'width': '100%', 'height': '600px'},
                                    srcDoc= sww.plot_heatmap(top_10, heat_df).to_html()
                                )
                            ]),
                            html.Div([
                                html.Iframe(
                                    id='mostworn2',
                                    style={'border-width': '0', 'width': '100%', 'height': '600px'},
                                    srcDoc= sww.plot_cpw(worn_df).to_html()
                                )
                            ]),
                        dbc.CardFooter("This is the footer")
                    ]),    
                ])
            ), 
            dbc.Col(
                dbc.Card([
                    dbc.CardBody([
                        html.Div([
                            html.Iframe(
                                id='mostworn3',
                                style={'border-width': '0', 'width': '100%', 'height': '600px'},
                                srcDoc = sww.plot_mostworn(worn_df).to_html()
                                )
                        ]),
                    ])
                ])
            ),
        ]),
    ],
    fluid=True
)

if __name__ == '__main__':
    app.run_server(
        port = 8071,
        debug=False)

In [ ]:
from dash import Dash, html, dcc, Input, Output
import altair as alt
import dash_bootstrap_components as dbc
import pandas as pd
import numpy as np
import matplotlib as plt
from matplotlib.pyplot import figure

import sheworewhat as sww

closet = sww.closet_df()
worn_df = sww.complete_df(closet)
top_10, heat_df = sww.top_10_df()

app = Dash(__name__)
server = app.server

app.title = "She Wore What 2023"
app = Dash(__name__, external_stylesheets=[dbc.themes.LUX])
app.layout = dbc.Container(
    [
        html.Br(),
        dbc.Row(
            dbc.Col(
                html.B("She Wore What 2023"),
                    style={"font-weight": "bold", 
                       "color": "#000000", 
                      "font-size" : "150%"}
            )
        ),
        html.Br(),
        dbc.Row([
            dbc.Col([
                    html.P(
                    "Hi! My name is Jasmine and I tracked every single item of clothing I wore in 2023. "
                    "As a data scientist and sustainable fashion enthusiast, I thought "
                    "this would be a fun project to help me analyze my personal style"
                    "trends so I can make smarter decisions about my purchases in the future.",
                    )
            ], width={"size": 6})
        ]),
        dbc.Row([
            dbc.Col([
                    html.Div([
                        html.Iframe(
                            id='top10',
                            style={'border-width': '0', 'width': '100%', 'height': '600px'},
                            srcDoc= sww.plot_mostworn(worn_df).to_html()
                        ),
                    ]),
            ]),
            dbc.Col([
                    html.P(
                    "To the left, we can see my top 10 most worn items. Some of these seem"
                    " obvious -- my Hollister puffer coat is my go-to winter coat. My gold brand "
                    "hoop earrings are a daily staple and part of my personal brand. But this plot"
                    " really doesn't say much -- it might be better to break down the items by category."
                    "Note to self: would be cool to insert images here", 
                    )
            ])
        ]),
        dbc.Row([
            dbc.Col([
                html.Div([
                    html.Iframe(
                        id='mostworn3',
                        style={'border-width': '0', 'width': '100%', 'height': '600px'},
                        srcDoc = sww.plot_heatmap(top_10, heat_df).to_html()
                        )
                ]),
            ]),
        ])
        ],
    fluid=True
)

if __name__ == '__main__':
    app.run_server(
        port = 8077,
        debug=False)

In [ ]:
from dash import Dash, html, dcc, Input, Output
import altair as alt
import dash_bootstrap_components as dbc
import pandas as pd
import numpy as np


import sheworewhat as sww

closet = sww.closet_df()
worn_df = sww.complete_df(closet)
top_10, heat_df = sww.top_10_df()

app = Dash(__name__)
server = app.server

app.title = "She Wore What 2023"

app = Dash(__name__, external_stylesheets=[dbc.themes.LUX]) #QUARTZ OR LUX
app.layout = dbc.Container(
    [
        html.Br(),
        dbc.Row(
            dbc.Col(
                html.B("She Wore What 2023"),
                    style={"font-weight": "bold", 
                       "color": "#000000", 
                      "font-size" : "150%"}
            )
        ),
        html.Br(),
        dbc.Row([
            dbc.Col([
                    html.P(
                    "Hi! My name is Jasmine and I tracked every single item of clothing I wore in 2023. "
                    "As a data scientist and sustainable fashion enthusiast, I thought "
                    "this would be a fun project to help me analyze my personal style"
                    "trends so I can make smarter decisions about my purchases in the future.",
                    )
            ], width={"size": 6})
        ]),
        dbc.Row([
            dbc.Col([ #
                html.Div(
                    dbc.Accordion(
                        [
                            # top 10 plots + facet plots
                            dbc.AccordionItem([
                                dbc.Row([
                                    dbc.Col([
                                        html.Div([
                                            html.Iframe(
                                                id='top10',
                                                style={'border-width': '0', 
                                                       'width': '100%', 'height': '400px'},
                                                srcDoc= sww.plot_mostworn(worn_df).to_html()
                                            ),
                                        ]),
                                    ]),
                                    dbc.Col([
                                        html.Br(), 
                                        html.Br(),
                                        html.P("My top 10 most worn items of the year. Some of these seem"
                                            " obvious -- my Hollister puffer coat is my go-to winter coat. My gold brand "
                                            "hoop earrings are a daily staple. But this plot doesn't tell us much on it's own"
                                            "let's break down the items by category. "
                                            "Note to self: would be cool to insert images here"
                                        )
                                    ]),
                                ]),
                                dbc.Row([
                                    dbc.Col([
                                        html.Br(),
                                        html.P("Facets!")
                                    ]),
                                    dbc.Col([
                                       html.Div([
                                            html.Iframe(
                                                id='top5_cat',
                                                style={'border-width': '0', 
                                                       'width': '100%', 'height': '800px'},
                                                srcDoc= sww.plot_facet(worn_df).to_html()
                                            ),
                                        ]), 
                                    ])
                                ]),
                            ],
                                title="Most Worn Items Overall",
                            ),
                            dbc.AccordionItem(
                                [
                                dbc.Row([
                                    dbc.Col([
                                        html.Br(),
                                        html.Br(),
                                        html.P("Fun little heatmap of the top 10 items -- need to work on drop down menu."
                                        )
                                    ]),
                                    dbc.Col([
                                        html.Div([
                                            html.Iframe(
                                                id='heatmap',
                                                style={'border-width': '0', 'width': '100%', 'height': '300px'},
                                                srcDoc = sww.plot_heatmap(top_10, heat_df).to_html()
                                                )
                                        ]),  
                                    ]),
                                ]),
                                ], title="Item Heatmap Throughout 2023"
                            ),
                            # cost per wear
                            dbc.AccordionItem(
                                [
                                    dbc.Row([
                                        dbc.Col([
                                            html.P("Cost-per-wear: price of item / number of time worn" 
                                                   "Please note, the cost-per-wear was only calculated for items for which price"
                                                   "was known, including items purchased secondhand. ")
                                        ]),
                                        dbc.Col([
                                            html.Div([
                                                html.Iframe(
                                                    id='costperwear',
                                                    style={'border-width': '0', 'width': '100%', 'height': '400px'},
                                                    srcDoc = sww.plot_cpw(worn_df).to_html()
                                                )
                                            ]),
                                        ]),
                                    ])
                                ],
                                title="Cost Per Wear",
                            ),
                            dbc.AccordionItem(
                                "This is the content of the third section",
                                title="Item 3",
                            ),
                        ], start_collapsed=True
                    )
                )    
            ]), 
        ])
        ],
    fluid=True
)

if __name__ == '__main__':
    app.run_server(
        port = 8078,
        debug=False)


Rearrange the accordion / omit boring plots

In [ ]:
from dash import Dash, html, dcc, Input, Output
import altair as alt
import dash_bootstrap_components as dbc
import pandas as pd
import numpy as np


import sheworewhat as sww

closet = sww.closet_df()
worn_df = sww.complete_df(closet)
top_10, heat_df = sww.top_10_df()

app = Dash(__name__)
server = app.server

app.title = "She Wore What 2023"

app = Dash(__name__, external_stylesheets=[dbc.themes.LUX]) #QUARTZ OR LUX
app.layout = dbc.Container(
    [
        html.Br(),
        dbc.Row(
            dbc.Col(
                html.B("She Wore What 2023"),
                    style={"font-weight": "bold", 
                       "color": "#000000", 
                      "font-size" : "150%"}
            )
        ),
        html.Br(),
        dbc.Row([
            dbc.Col([
                    html.P(
                    "Hi! My name is Jasmine and I tracked every single item of clothing I wore in 2023. "
                    "As a data scientist and sustainable fashion enthusiast, I thought "
                    "this would be a fun project to help me analyze my personal style"
                    "trends so I can make smarter decisions about my purchases in the future.",
                    )
            ], width={"size": 6})
        ]),
        dbc.Row([
            dbc.Col([ 
                html.Div(
                    dbc.Accordion(
                        [
                            dbc.AccordionItem(
                                [
                                    dbc.Row([
                                        dbc.Col([
                                            html.Br(),
                                            html.Br(),
                                            html.P("The first thing I did was log everything in my closet. "
                                                   "A tedious process, but in the end I had {y} pieces."
                                                   "The majority of my closet is black or white."
                                                   "Colors represented in my closet -- need to add secondary colors.")
                                           ], width={"size": 6, "offset": 3}
                                        ),
                                    dbc.Row([
                                        dbc.Col([
                                            html.P("In 2023, I added {x} new items to my closet. Of these items, "
                                                  "__% of the items were pre-loved (obtained through secondhand "
                                                   "stores or hand me down. It's my goal for 90% of my closet to be secondhand!")
                                        ], width={"offset": 3}),
#                                         dbc.Col[(
#                                                 html.H4("Well done!")
#                                         )],
                                        dbc.Col([
                                            html.Div([
                                                html.Iframe(
                                                    id="new-items",
                                                    style={'border-width': '0', 'width': '100%', 'height': '400px'},
                                                    srcDoc = sww.plot_color(worn_df).to_html()
                                                )
                                            ])
                                        ])
                                     ]),
                                         dbc.Col([
                                           html.Div([
                                                html.Iframe(
                                                    id="color",
                                                    style={'border-width': '0', 'width': '100%', 'height': '400px'},
                                                    srcDoc = sww.plot_color(worn_df).to_html()
                                                )
                                            ])
                                        ]),
                                    ]),
                                    dbc.Row([
                                        dbc.Col([
                                            html.Br(),
                                            html.Div([
                                                html.Iframe(
                                                    id="catwheel",
                                                    style={'border-width': '0', 'width': '100%', 'height': '400px'},
                                                    srcDoc = sww.plot_categories(worn_df).to_html()
                                                )
                                            ])
                                        ]),
                                        dbc.Col([
                                            html.Br(),
                                            html.Br(),
                                            html.Br(),
                                            html.Br(),
                                            html.P("I own a lot of tops and very few coats.")
                                        ], width={"size": 3}),
                                        dbc.Col([
                                            html.Br(),
                                            html.Br(),
                                            html.Br(),
                                           html.Div([
                                                html.Iframe(
                                                    id="bought",
                                                    style={'border-width': '0', 'width': '100%', 'height': '400px'},
                                                    srcDoc = sww.plot_bought(worn_df).to_html()
                                                )
                                            ])
                                        ])
                                    ]),
                                ],
                                title = "Wardrobe Analysis"
                            ),
                            dbc.AccordionItem(
                                [
                                dbc.Row([
                                    dbc.Col([
                                        html.P("My top 10 most worn items ..."
                                        )
                                    ]),
                                ]),
                                dbc.Row([
                                    dbc.Col([
                                        html.Div([
                                            html.Iframe(
                                                id='top10',
                                                style={'border-width': '0', 'width': '100%', 'height': '300px'},
                                                srcDoc = sww.plot_mostworn(worn_df).to_html()
                                                )
                                        ]),
                                    ], width={"size": 5}),
                                    dbc.Col([
                                        html.Div([
                                            html.Iframe(
                                                id='heatmap',
                                                style={'border-width': '0', 'width': '100%', 'height': '300px'},
                                                srcDoc = sww.plot_heatmap(top_10, heat_df).to_html()
                                                )
                                        ]),  
                                    ]),
                                ]),
                                ], title="Most Worn Items of 2023"
                            ),
                            # cost per wear
                            dbc.AccordionItem(
                                [
                                    dbc.Row([
                                        dbc.Col([
                                            html.P("Cost-per-wear: price of item / number of time worn" 
                                                   "Please note, the cost-per-wear was only calculated for items for which price"
                                                   "was known, including items purchased secondhand. ")
                                        ]),
                                        dbc.Col([
                                            html.Div([
                                                html.Iframe(
                                                    id='costperwear',
                                                    style={'border-width': '0', 'width': '100%', 'height': '425px'},
                                                    srcDoc = sww.plot_cpw(worn_df).to_html()
                                                )
                                            ]),
                                        ]),
                                    ])
                                ],
                                title="Cost Per Wear",
                            ),
                            dbc.AccordionItem(
                                "This is the content of the third section",
                                title="Item 3",
                            ),
                        ], start_collapsed=True
                    )
                )    
            ]), 
        ])
        ],
    fluid=True
)

if __name__ == '__main__':
    app.run_server(
        port = 8076,
        debug=False)

ok try to make everything central -- it worked! but it;s ugly so reverting lol

In [ ]:
from dash import Dash, html, dcc, Input, Output
import altair as alt
import dash_bootstrap_components as dbc
import pandas as pd
import numpy as np


import sheworewhat as sww

closet = sww.closet_df()
worn_df = sww.complete_df(closet)
top_10, heat_df = sww.top_10_df()

app = Dash(__name__)
server = app.server

app.title = "She Wore What 2023"

app = Dash(__name__, external_stylesheets=[dbc.themes.LUX])
app.layout = dbc.Container(
    [
        html.Br(),
        dbc.Row(
            dbc.Col(
                html.B("She Wore What 2023"),
                    style={"font-weight": "bold", 
                       "color": "#000000", 
                      "font-size" : "150%"}
            )
        ),
        html.Br(),
        dbc.Row([
            dbc.Col([
                    html.P(
                    "Hi! My name is Jasmine and I tracked every single item of clothing I wore in 2023. "
                    "As a data scientist and sustainable fashion enthusiast, I thought "
                    "this would be a fun project to help me analyze my personal style"
                    "trends so I can make smarter decisions about my purchases in the future.",
                    )
            ], width={"size": 6, "offset" : 3})
        ]),
        dbc.Row([
            dbc.Col([ 
                html.Div(
                    dbc.Accordion(
                        [
                            dbc.AccordionItem(
                                [
                                    dbc.Row([
                                        dbc.Col([
                                            html.H4("The process"),
                                            html.P("The first thing I did was take stock of everything in my closet -- "
                                                   "all {x} pieces. The most popular colors in my closet were black and white"
                                                   " -- need to add secondary colors still and also these numbers dont add up (87 colors and 93 items?)."),
                                            html.Br()
                                           ], width={"size": 6, "offset": 3}
                                        ),
                                    ]),
                                    dbc.Row([
                                        dbc.Col([
                                            html.Div([
                                                html.Iframe(
                                                    id="color_composition",
                                                    style={'border-width': '0', 'width': '100%', 'height': '400px'},
                                                    srcDoc = sww.plot_color(worn_df).to_html()
                                                )
                                            ])
                                        ], width={"size": 6, "offset": 4})
                                     ]),
                                    dbc.Row([
                                        dbc.Col([
                                            html.Br(),
                                            html.Div([
                                                html.Iframe(
                                                    id="catwheel",
                                                    style={'border-width': '0', 'width': '100%', 'height': '400px'},
                                                    srcDoc = sww.plot_categories(worn_df).to_html()
                                                )
                                            ])
                                        ]),
                                        dbc.Col([
                                            html.Br(),
                                            html.Br(),
                                            html.Br(),
                                            html.Br(),
                                            html.P("There was a lot of interesting patterns to deduce from my existing closet."
                                                  "For example, almost all my tops are secondhands while all my pants were purchased brand-new.")
                                        ], width={"size": 3}),
                                        dbc.Col([
                                            html.Br(),
                                            html.Br(),
                                            html.Br(),
                                           html.Div([
                                                html.Iframe(
                                                    id="bought",
                                                    style={'border-width': '0', 'width': '100%', 'height': '400px'},
                                                    srcDoc = sww.plot_bought(worn_df).to_html()
                                                )
                                            ])
                                        ])
                                    ]),
                                    dbc.Row([
                                        dbc.Col([
                                            html.P("In 2023, I added {x} new items to my closet. Of these items, "
                                            "__% of the items were pre-loved (obtained through secondhand "
                                            "stores or hand me down. It's my goal for 90% of my closet to be secondhand!")
                                        ], width={"size" : 6, "offset" : 3})
                                    ]),
                                    dbc.Row([
                                        dbc.Col([
                                            html.Div([
                                                html.Iframe(
                                                    id="facet-items",
                                                    style={'border-width': '0', 'width': '100%', 'height': '400px'},
                                                    srcDoc = sww.plot_facet(worn_df).to_html()
                                                )
                                            ])
                                        ], width={"size": 6, "offset": 3})
                                     ]),
                                ],
                                title = "Wardrobe Analysis"
                            ),
                            dbc.AccordionItem(
                                [
                                dbc.Row([
                                    dbc.Col([
                                        html.H4("Top 10 Most Worn Items"
                                        )
                                    ]),
                                ]),
                                dbc.Row([
                                    dbc.Col([
                                        html.Div([
                                            html.Iframe(
                                                id='top10',
                                                style={'border-width': '0', 'width': '100%', 'height': '300px'},
                                                srcDoc = sww.plot_mostworn(worn_df).to_html()
                                                )
                                        ]),
                                    ], width={"size": 5}),
                                    dbc.Col([
                                        html.Div([
                                            html.Iframe(
                                                id='heatmap',
                                                style={'border-width': '0', 'width': '100%', 'height': '300px'},
                                                srcDoc = sww.plot_heatmap(top_10, heat_df).to_html()
                                                )
                                        ]),  
                                    ]),
                                ]),
                                dbc.Row([
                                    dbc.Col([
                                        html.P("My most worn pieces of clothing really ran the gamut. I'm not surprised that my most worn item"
                                               " so far is my green Hollister Puffer jacket. Winter was especially brutal in 2023, which also "
                                               "explains that my second most worn piece is my Blondo waterproof boots!"
                                               ),
                                        html.P("While this data is interesting, it's too general to learn which items are considered core staples."
                                              " Let's take a peak at the most worn items per category (tops, bottoms, shoes) instead!")
                                    ])
                                ]),
                                dbc.Row([
                                        dbc.Col([
                                            html.Div([
                                                html.Iframe(
                                                    id="new-items",
                                                    style={'border-width': '0', 'width': '100%', 'height': '700px'},
                                                    srcDoc = sww.plot_facet(worn_df).to_html()
                                                )
                                            ])
                                        ], width={"size" : 6}),
                                        dbc.Col([
                                            html.Br(),
                                            html.Br(),
                                            html.P("Super interesting insightful data insights")
                                        ])
                                     ]),
                                ], title="Most Worn Items of 2023"
                            ),
                            # cost per wear
                            dbc.AccordionItem(
                                [
                                    dbc.Row([
                                        dbc.Col([
                                            html.H4("Cost-per-wear"),
                                            html.P("Cost-per-wear: price of item / number of times worn in a single year"),
                                            html.Br(),
                                            html.P("Note: cost-per-wear was only calculated for items for which the price"
                                                   "was known, including items purchased secondhand. ")
                                        ]),
                                        dbc.Col([
                                            html.Div([
                                                html.Iframe(
                                                    id='costperwear',
                                                    style={'border-width': '0', 'width': '100%', 'height': '425px'},
                                                    srcDoc = sww.plot_cpw(worn_df).to_html()
                                                )
                                            ]),
                                        ]),
                                    ])
                                ],
                                title="Cost Per Wear",
                            ),
                            dbc.AccordionItem(
                                html.P("This would be a fun section to look at winter/spring/fall/summer trends. "
                                    "Unfortunately, we are only 15 days into winter so the data is not there yet."
                                ),
                                title="Seasonal Trends",
                            ),
                        ], start_collapsed=True
                    )
                )    
            ]), 
        ])
        ],
    fluid=True
)

if __name__ == '__main__':
    app.run_server(
        port = 8076,
        debug=False)

# ok reverting back to non-central column

In [1]:
from dash import Dash, html, dcc, Input, Output
import altair as alt
import dash_bootstrap_components as dbc
import pandas as pd
import numpy as np


import sheworewhat as sww

closet = sww.closet_df()
worn_df = sww.worn(closet)
top_id, top_item, heat_df = sww.top_10_df()

app = Dash(__name__)
server = app.server

app.title = "She Wore What 2023"

app = Dash(__name__, external_stylesheets=[dbc.themes.LUX])
app.layout = dbc.Container(
    [
        html.Br(),
        dbc.Row(
            dbc.Col(
                html.B("She Wore What 2023"),
                    style={"font-weight": "bold", 
                       "color": "#000000", 
                      "font-size" : "150%"},
                width={"offset" : 5}
            )
        ),
        html.Br(),
        dbc.Row([
            dbc.Col([
                    html.P(
                    "Hi! My name is Jasmine and I tracked every single item of clothing I wore in 2023. "
                    "As a data scientist and sustainable fashion enthusiast, I thought "
                    "this would be a fun project to help me analyze my personal style"
                    "trends so I can make smarter decisions about my purchases in the future.",
                    )
            ])
        ]),
        dbc.Row([
            dbc.Col([ 
                html.Div(
                    dbc.Accordion(
                        [
                            dbc.AccordionItem(
                                [
                                    dbc.Row([
                                        dbc.Col([
                                            html.H4("The process"),
                                            html.P("The first thing I did was take stock of everything in my closet -- "
                                                   "all {x} pieces. The most popular colors in my closet were black and white"
                                                   " -- need to add secondary colors still and also these numbers dont add up (87 colors and 93 items?)."),
                                            html.Br()
                                           
                                        ]),
                                        dbc.Col([
                                            html.Div([
                                                html.Iframe(
                                                    id="color_composition",
                                                    style={'border-width': '0', 'width': '100%', 'height': '400px'},
                                                    srcDoc = sww.plot_color(worn_df).to_html()
                                                )
                                            ])
                                        ], width={"size": 4})
                                     ]),
                                    dbc.Row([
                                        dbc.Col([
                                            html.Br(),
                                            html.Div([
                                                html.Iframe(
                                                    id="catwheel",
                                                    style={'border-width': '0', 'width': '100%', 'height': '400px'},
                                                    srcDoc = sww.plot_categories(worn_df).to_html()
                                                )
                                            ])
                                        ]),
                                        dbc.Col([
                                            html.Br(),
                                            html.Br(),
                                            html.Br(),
                                            html.Br(),
                                            html.P("There was a lot of interesting patterns to deduce from my existing closet."
                                                  "For example, almost all my tops are secondhands while all my pants were purchased brand-new.")
                                        ], width={"size": 8}),
                                    ]),
                                    dbc.Row([
                                        dbc.Col([
                                            html.P("In 2023, I added {x} new items to my closet. Of these items, "
                                            "__% of the items were pre-loved (obtained through secondhand "
                                            "stores or hand me down.")
                                        ], width={"size" : 6}),
                                        dbc.Col([
                                            html.Div([
                                                html.Iframe(
                                                    id="facet-items",
                                                    style={'border-width': '0', 'width': '100%', 'height': '400px'},
                                                    srcDoc = sww.plot_facet(worn_df).to_html()
                                                )
                                            ])
                                        ])
                                     ]),
                                    dbc.Row([
                                        dbc.Col([
                                           html.Div([
                                                html.Iframe(
                                                    id="bought",
                                                    style={'border-width': '0', 'width': '100%', 'height': '400px'},
                                                    srcDoc = sww.plot_bought(worn_df).to_html()
                                                )
                                            ])
                                        ]),
                                        dbc.Col([
                                            html.Br(),
                                            html.Br(),
                                            html.P("I try my best to reduce buying 'new' clothing items as much as possible. "
                                                   "For ethical and sustainability reasons, I aspire for my closet for "
                                                   "the majority of my closet to be secondhand! "
                                                  "If you'd like to learn more about the exploitative labor practices of fast fashion, "
                                                  "I highly recommend listening to Binchtopia's 'SheInvestigation' episode, which inspired me to "
                                                  "track my own personal style patterns so I can spend more wisely.")
                                        ])
                                    ])
                                ],
                                title = "Wardrobe Analysis"
                            ),
                            dbc.AccordionItem(
                                [
                                dbc.Row([
                                    dbc.Col([
                                        html.H4("Top 10 Most Worn Items"
                                        )
                                    ]),
                                ]),
                                dbc.Row([
                                    html.P("My most worn pieces of clothing really ran the gamut. I'm not surprised that my most worn item"
                                               " so far is my green Hollister Puffer jacket. Winter was especially brutal in 2023, which also "
                                               "explains that my second most worn piece is my Blondo waterproof boots!"
                                               )
                                ]),
                                dbc.Row([
                                    dbc.Col([
                                        html.Div([
                                            html.Iframe(
                                                id='top10',
                                                style={'border-width': '0', 'width': '100%', 'height': '300px'},
                                                srcDoc = sww.plot_mostworn(worn_df).to_html()
                                                )
                                        ]),
                                    ], width={"size": 5}),
                                    dbc.Col([
                                        html.Div([
                                            dcc.Dropdown(
                                                id='select-item',
                                                options=[{'label': item, 'value': i} for i, item in enumerate(top_item)]), 
                                            html.Iframe(
                                                id='heatmap_item',
                                                style={'border-width': '0', 'width': '100%', 'height': '400px'},
                                                srcDoc=sww.plot_heatmap(top_id, heat_df).to_html())
#                                             html.Iframe(
#                                                 id='heatmap',
#                                                 style={'border-width': '0', 'width': '100%', 'height': '300px'},
#                                                 srcDoc = sww.plot_heatmap(top_id, heat_df).to_html()
#                                                 )
                                        ]),  
                                    ]),
                                ]),
                                dbc.Row([
                                    dbc.Col([
                                        html.P("My most worn pieces of clothing really ran the gamut. I'm not surprised that my most worn item"
                                               " so far is my green Hollister Puffer jacket. Winter was especially brutal in 2023, which also "
                                               "explains that my second most worn piece is my Blondo waterproof boots!"
                                               ),
                                        html.P("While this data is interesting, it's too general to learn which items are considered core staples."
                                              " Let's take a peak at the most worn items per category (tops, bottoms, shoes) instead!")
                                    ])
                                ]),
                                dbc.Row([
                                        dbc.Col([
                                            html.Div([
                                                html.Iframe(
                                                    id="new-items",
                                                    style={'border-width': '0', 'width': '100%', 'height': '700px'},
                                                    srcDoc = sww.plot_facet(worn_df).to_html()
                                                )
                                            ])
                                        ], width={"size" : 6}),
                                        dbc.Col([
                                            html.Br(),
                                            html.Br(),
                                            html.P("Super interesting insightful data insights")
                                        ])
                                     ]),
                                ], title="Most Worn Items of 2023"
                            ),
                            # cost per wear
                            dbc.AccordionItem(
                                [
                                    dbc.Row([
                                        dbc.Col([
                                            html.H4("Cost-per-wear"),
                                            html.P("Cost-per-wear: price of item / number of times worn in a single year"),
                                            html.Br(),
                                            html.P("Note: cost-per-wear was only calculated for items for which the price"
                                                   "was known, including items purchased secondhand. ")
                                        ]),
                                        dbc.Col([
                                            html.Div([
                                                html.Iframe(
                                                    id='costperwear',
                                                    style={'border-width': '0', 'width': '100%', 'height': '425px'},
                                                    srcDoc = sww.plot_cpw(worn_df).to_html()
                                                )
                                            ]),
                                        ]),
                                    ])
                                ],
                                title="Cost Per Wear",
                            ),
                            dbc.AccordionItem(
                                html.P("This would be a fun section to look at winter/spring/fall/summer trends. "
                                    "Unfortunately, we are only 15 days into winter so the data is not there yet."
                                ),
                                title="Seasonal Trends",
                            ),
                        ], start_collapsed=True
                    )
                )    
            ]), 
        ])
        ],
    fluid=True
)

@app.callback(
    Output('heatmap_item', 'srcDoc'),
    Input('select-item', 'value'))

def update_output(z):
    return sww.plot_heatmap(top_id, df, z).to_html()


if __name__ == '__main__':
    app.run_server(
        port = 8078,
        debug=False)

Dash is running on http://127.0.0.1:8078/

Dash is running on http://127.0.0.1:8078/

 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 8078 is in use by another program. Either identify and stop that program, or start the server with a different port.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/jasmineortega/opt/miniconda3/lib/python3.9/site-packages/werkzeug/serving.py", line 911, in prepare_socket
    s.bind(server_address)
OSError: [Errno 48] Address already in use

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/jasmineortega/opt/miniconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/qh/6chp8sds4c7gybyg8m5n7k3m0000gn/T/ipykernel_39209/1045406944.py", line 246, in <module>
    app.run_server(
  File "/Users/jasmineortega/opt/miniconda3/lib/python3.9/site-packages/dash/dash.py", line 2133, in run_server
    self.run(*args, **kwargs)
  File "/Users/jasmineortega/opt/miniconda3/lib/python3.9/site-packages/dash/dash.py", line 1980, in run
    self.server.run(host=host, port=port, debug=debug, **flask_run_options)
  File "/Users/jasmineortega/o

TypeError: object of type 'NoneType' has no len()